importing packages

In [1]:
import pandas as pd, requests, numpy as np

Part 1 - getting time series for the cryptocurrencies (ETH,SOL,AVAX,USDT,FLOW)from cryptowat.ch starting from 2021-11-22, hourly data

In [2]:
def getData(token):
    if token == 'FLOW':
        exchange = 'kraken'
    else:
        exchange = 'coinbase-pro'
    response = requests.get(f'https://api.cryptowat.ch/markets/{exchange}/{token}usd/ohlc',params={'periods': '3600','after': str(int(pd.Timestamp('2021-11-22').timestamp()))},timeout=10)
    df = pd.DataFrame(response.json()['result']['3600'],columns=['ts','open','high','low','close','volume','volumeUSD'])
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    return df


In [3]:
tokenList = ['ETH','SOL','AVAX','USDT','FLOW']
dfs = [getData(t) for t in tokenList]
data = pd.concat(dfs)
data.sort_values(['ts','token'],inplace=True)   
data.head(10)

,ts,open,high,low,close,volume,volumeUSD,token
0,2021-11-22 00:00:00,131.6900,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX
0,2021-11-22 00:00:00,4317.9800,4342.2400,4246.0700,4262.9900,7.262563e+03,3.123077e+07,ETH
0,2021-11-22 00:00:00,13.8870,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW
0,2021-11-22 00:00:00,230.4830,234.6880,228.8050,230.7850,9.353180e+04,2.172284e+07,SOL
0,2021-11-22 00:00:00,1.0010,1.0017,1.0009,1.0012,9.987681e+06,9.999090e+06,USDT
1,2021-11-22 01:00:00,128.7600,135.2900,126.2500,134.8500,1.471799e+05,1.915462e+07,AVAX
1,2021-11-22 01:00:00,4263.0400,4270.3400,4212.4500,4234.3700,8.437946e+03,3.574586e+07,ETH
1,2021-11-22 01:00:00,14.0530,14.4530,13.5430,13.6210,2.910879e+04,4.117675e+05,FLOW
1,2021-11-22 01:00:00,230.7520,232.2690,225.0520,226.0000,8.884100e+04,2.017188e+07,SOL
1,2021-11-22 01:00:00,1.0012,1.0014,1.0012,1.0013,7.766318e+06,7.775823e+06,USDT


Part 2 - Get the total USD volume traded for each token in a dataframe, sorted from highest volume to lowest volume

In [4]:
data.groupby('token')['volumeUSD'].sum().sort_values(ascending=False)

token
ETH     2.615216e+10
SOL     8.546535e+09
AVAX    4.754746e+09
USDT    2.358773e+09
FLOW    1.002591e+08
Name: volumeUSD, dtype: float64

Part 3 - Add a column that calculates the close price ratio between ETH and SOL for each house (i.e. close price of ETH / close price of SOL for each period)

In [5]:
eth = data[data.token=="ETH"].set_index("ts")
sol = data[data.token=="SOL"].set_index("ts")

CPRATIO = (eth.close / sol.close).to_frame("ETH/SOL_CP")
data = data.merge(CPRATIO,on="ts",how="left")
data.head(5)

,ts,open,high,low,close,volume,volumeUSD,token,ETH/SOL_CP
0,2021-11-22,131.690,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.471694
1,2021-11-22,4317.980,4342.2400,4246.0700,4262.9900,7.262563e+03,3.123077e+07,ETH,18.471694
2,2021-11-22,13.887,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.471694
3,2021-11-22,230.483,234.6880,228.8050,230.7850,9.353180e+04,2.172284e+07,SOL,18.471694
4,2021-11-22,1.001,1.0017,1.0009,1.0012,9.987681e+06,9.999090e+06,USDT,18.471694


Part 4 - Change the name of the volume and volumeUSD columns to volumeBase and volumeTerm

In [6]:
data.rename(columns={"volume":"volumeBase","volumeUSD":"volumeTerm"},inplace=True)
data.head(5)

,ts,open,high,low,close,volumeBase,volumeTerm,token,ETH/SOL_CP
0,2021-11-22,131.690,133.7900,127.7000,128.6700,9.686748e+04,1.267080e+07,AVAX,18.471694
1,2021-11-22,4317.980,4342.2400,4246.0700,4262.9900,7.262563e+03,3.123077e+07,ETH,18.471694
2,2021-11-22,13.887,14.1690,13.8870,14.1500,2.326138e+04,3.274755e+05,FLOW,18.471694
3,2021-11-22,230.483,234.6880,228.8050,230.7850,9.353180e+04,2.172284e+07,SOL,18.471694
4,2021-11-22,1.001,1.0017,1.0009,1.0012,9.987681e+06,9.999090e+06,USDT,18.471694


Part 5 - create a fat table indexed by the timestamp, and each column is the close price of each token (i.e. this should be a table of 200 rows and 5 columns)

In [7]:
closePrice = data.set_index(["ts","token"])["close"]
closePrice = closePrice.unstack(level=1)
closePrice.head(5)

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,128.67,4262.99,14.150,230.785,1.0012
2021-11-22 01:00:00,134.85,4234.37,13.621,226.000,1.0013
2021-11-22 02:00:00,132.54,4217.89,13.753,227.037,1.0012
2021-11-22 03:00:00,133.38,4193.47,13.652,223.117,1.0011
2021-11-22 04:00:00,136.29,4168.35,13.581,220.652,1.0009


Part 6 - calculate the hour by hour log return of the close price of each token (return is calculated by np.log(price_t / price_{t-1}))

In [8]:
returnDf = closePrice / closePrice.shift()
returnDf = np.log(returnDf)
returnDf.head(5)

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,NaN,NaN,NaN,NaN,NaN
2021-11-22 01:00:00,0.046912,-0.006736,-0.038102,-0.020952,0.0001
2021-11-22 02:00:00,-0.017279,-0.003900,0.009644,0.004578,-0.0001
2021-11-22 03:00:00,0.006318,-0.005806,-0.007371,-0.017417,-0.0001
2021-11-22 04:00:00,0.021583,-0.006008,-0.005214,-0.011109,-0.0002


Part 7 - [Stretch] calculate the correlation of the tokens using the table above

In [9]:
corrDf = returnDf.corr()
corrDf

token,AVAX,ETH,FLOW,SOL,USDT
token,,,,,
AVAX,1.000000,0.659412,0.405564,0.648623,0.016597
ETH,0.659412,1.000000,0.579961,0.761419,-0.085039
FLOW,0.405564,0.579961,1.000000,0.478228,-0.174387
SOL,0.648623,0.761419,0.478228,1.000000,0.103927
USDT,0.016597,-0.085039,-0.174387,0.103927,1.000000


Part 8 - [Stretch] visualize the correlation in a matplpotlib plot

In [10]:
corrDf.style.background_gradient(cmap='Greens')

token,AVAX,ETH,FLOW,SOL,USDT
token,,,,,
AVAX,1.000000,0.659412,0.405564,0.648623,0.016597
ETH,0.659412,1.000000,0.579961,0.761419,-0.085039
FLOW,0.405564,0.579961,1.000000,0.478228,-0.174387
SOL,0.648623,0.761419,0.478228,1.000000,0.103927
USDT,0.016597,-0.085039,-0.174387,0.103927,1.000000
